# This notebook will solve the MDP by calling the policy iteration functions but using a probabilistic action set

In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import Policy_iteration_functions as f
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import winsound

%matplotlib qt5

## Base transition matrix and values 
Create the stochastic action set and create the transition matrix and solve for the V's

In [2]:
count = pd.read_pickle(Path.cwd() / 'pickle' / 'TP' /'Counts_0.8_doubleprime.plk')

q_ad_serve, q_deuce_serve, q_rally = f.get_q_dist()
T = f.get_probabilistic_transition_matrix(q_ad_serve, q_deuce_serve, q_rally)
V_base = f.calculate_V(T)
V_sum, V_avg, V_top_avg, V_serve, V_count_worse = f.get_V_measures(V_base,count,V_base)
V_full = dm.reformatVector(np.reshape(V_base, (-1, 1)))
count_full = dm.reformatVector(count.to_numpy())

## Error Scenario #1: Short Shorts Erring Long

In [ ]:
error_fracs = [0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]

V_stats_short_err = pd.DataFrame(np.zeros((len(error_fracs),5)), columns=['V_sum', 'V_avg', 'V_top_avg', 'V_serve', 'V_count_worse'], index = error_fracs)

for ix,err in enumerate(error_fracs):
    q_ad_serve, q_deuce_serve, q_rally = f.get_q_dist()
    
    # make copy of short shots and get err % of them
    adj = q_rally.iloc[:,0:3].copy()
    adj *= err 
    # subtract the err % from short shots
    q_rally.iloc[:,0:3] -= err*q_rally.iloc[:,0:3]
    # Add the err % to the long shots 
    q_rally.iloc[:,3] += adj.iloc[:,0]
    q_rally.iloc[:,4] += adj.iloc[:,1]
    q_rally.iloc[:,5] += adj.iloc[:,2]
    
    # Create the probabilistic action T and solve for V's
    T = f.get_probabilistic_transition_matrix(q_ad_serve, q_deuce_serve, q_rally)
    V = f.calculate_V(T)
    
    # Calculate the summary stats
    V_stats_short_err.iloc[ix,:] = f.get_V_measures(V,count,V_base)

winsound.Beep(2000,800)

In [ ]:
f.plot_error_values(V_stats_short_err,"Impact of Short Shots Erring Long",3)

## Error Scenario #1: Long Shorts Erring Short

In [ ]:
error_fracs = [0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]

V_stats_long_err = pd.DataFrame(np.zeros((len(error_fracs),5)), columns=['V_sum', 'V_avg', 'V_top_avg', 'V_serve', 'V_count_worse'], index = error_fracs)

for ix,err in enumerate(error_fracs):
    q_ad_serve, q_deuce_serve, q_rally = f.get_q_dist()
    
    # make copy of short shots and get err % of them
    adj = q_rally.iloc[:,3:6].copy()
    adj *= err 
    # subtract the err % from short shots
    q_rally.iloc[:,3:6] -= err*q_rally.iloc[:,3:6]
    # Add the err % to the long shots 
    q_rally.iloc[:,0] += adj.iloc[:,0]
    q_rally.iloc[:,1] += adj.iloc[:,1]
    q_rally.iloc[:,2] += adj.iloc[:,2]
    
    # Create the probabilistic action T and solve for V's
    T = f.get_probabilistic_transition_matrix(q_ad_serve, q_deuce_serve, q_rally)
    V = f.calculate_V(T)
    
    # Calculate the summary stats
    V_stats_long_err.iloc[ix,:] = f.get_V_measures(V,count, V_base)

winsound.Beep(2000,800)

In [ ]:
f.plot_error_values(V_stats_long_err,"Impact of Long Shots Erring Short",3)

## Error Scenario #3: Forehand / Backhand error
For the bottom four rows of rally shots, take fraction of the proportion
of the 2nd most chosen action, and put it to the most chosen action

In [9]:
error_fracs = [0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]
V_stats_forehand_err = pd.DataFrame(np.zeros((len(error_fracs),5)), columns=['V_sum', 'V_avg', 'V_top_avg', 'V_serve', 'V_count_worse'], index = error_fracs)

for ix,err in enumerate(error_fracs):
    q_ad_serve, q_deuce_serve, q_rally = f.get_q_dist()
    for i in range(1764,5297):
        
        shot_type = f.check_forehand_vs_backhand(i)
        
        if shot_type == 'forehand':
        
            # Get 2 actions that are most used and calculate err from the max
            top_2 = q_rally.iloc[i].nlargest(2)
            adj = top_2.iloc[0]*err
            
            # Remove adj from max and add it to the second_max
            q_rally.iloc[i,top_2.index[0]] -= adj
            q_rally.iloc[i,top_2.index[1]] += adj
        
    # Create the probabilistic action T and solve for V's
    T = f.get_probabilistic_transition_matrix(q_ad_serve, q_deuce_serve, q_rally)
    V = f.calculate_V(T)
    
    # Calculate the summary stats
    V_stats_forehand_err.iloc[ix,:] = f.get_V_measures(V,count, V_base)
    
winsound.Beep(2000,800)

In [8]:
f.plot_error_values(V_stats_forehand_err,"Impact of Forehand Error",3)

In [5]:
bucket = 27
choice = 'rally'

plt.figure()
pf.createMRPheatmap(V_full, count_full, bucket, choice,vmin=0, compressed=True, only_half=False, big_buckets=True, half_label=True)
winsound.Beep(2000,800)